# Program to rate characters
Create a model based on a persons appeal to different characters in Genshin Impact
Following this form: https://forms.office.com/Pages/ResponseDetailPage.aspx?id=tTAKZi6OaUe560ryi_0SvUHGZGPvcItElIw3gzwgl7NUMFJZUE5aNDVORVVMRENTSzFQV0laTjc4Vy4u&rid=4&GetResponseToken=q7rahzF2E7Igy2bHzBfrQBctplWW1STeVmCDaCKbsME&origin=rc

#### inputs

In [30]:
import torch as pt
from torch import nn

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

device = "cuda" if pt.cuda.is_available() else "cpu"



#### importing the data

In [108]:
from sys import path

InputData = pd.read_csv(path[0]+'/db.csv').drop(columns='Name')
InputData = InputData.to_numpy(dtype=int)

TotalOutputData = pd.read_csv(path[0]+'/rating.csv').drop(columns='Name')

Chosen_person = 'Gustav'
OutputData = TotalOutputData[Chosen_person].to_numpy(dtype=int)

print(InputData[0])
print(OutputData[0])

#del TotalOutputData

[0 3 5 1 0 0 0 0 0 1 0 0]
4


In [109]:
# analyse the data
print(f'Mean: {OutputData.mean()}')
print(f'Max: {OutputData.max()}')
print(f'Min: {OutputData.min()}')

vals, counts = np.unique(OutputData, return_counts=True)
mode_value = np.argwhere(counts == np.max(counts))
print(f'Mode: {mode_value.flatten()}') # if there are two values there are equally many of both
del vals, counts, mode_value

Mean: 4.9879518072289155
Max: 9
Min: 1
Mode: [3 4]


#### Splitting the data

In [110]:
from sklearn.model_selection import train_test_split

input_train, input_test, output_train, output_test = train_test_split(InputData, OutputData, test_size=0.2, random_state=42)


print(input_test, "\n"*2, output_test)

[[0 2 7 1 1 0 0 0 0 1 0 1]
 [0 3 5 1 0 0 0 0 0 1 0 0]
 [1 2 1 0 1 0 0 0 0 1 1 0]
 [0 3 5 1 0 1 0 0 0 0 0 1]
 [0 2 6 1 0 0 0 0 1 0 0 1]
 [0 2 4 1 0 0 0 0 0 0 0 1]
 [1 2 1 0 0 1 0 0 0 1 0 0]
 [1 2 4 0 0 0 0 0 0 1 0 0]
 [1 2 1 0 0 0 1 0 0 0 0 0]
 [1 2 1 0 0 1 0 0 0 1 0 0]
 [0 2 1 0 0 1 0 0 0 0 0 1]
 [0 2 3 1 0 1 0 0 0 0 0 0]
 [1 3 1 0 0 1 0 0 0 0 0 0]
 [0 2 3 1 1 0 0 0 0 0 0 0]
 [1 3 6 0 0 0 0 0 0 0 0 1]
 [1 3 1 0 0 0 0 0 0 1 0 0]
 [0 3 8 1 1 0 0 0 0 0 0 1]] 

 [3 4 5 4 4 5 8 6 9 7 6 6 5 4 5 4 5]


#### Building a model

In [111]:
from torch import nn
pt.manual_seed(42)

class ModelV0(nn.Module):
    def __init__(self, input_features:int = 12, output_features:int = 1, hidden_units:int = 10):
        super().__init__()
        self.Layer_1 = nn.Linear(in_features=input_features, out_features=hidden_units)
        self.layer_2 = nn.Linear(in_features=hidden_units, out_features=output_features)
    def forward(self, x):
        return self.layer_2(self.Layer_1(x))

In [112]:
model0 = ModelV0()

In [113]:
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss has tha sigmoid activation function built in
 
optimizer = pt.optim.SGD(params=model0.parameters(), lr=0.001)

In [114]:
def standard_deviation(y_true, y_preds):
    return pt.std_mean(pt.abs(y_true-y_preds))